In [1]:
import plotly.graph_objects as go
import json
import pandas as pd

from utils import calculate_nutripoints

In [2]:
# Geojson data source:
# https://github.com/ft-interactive/geo-data/blob/master/uk/london/london-wards-2014.geojson

GEO_DATA_PATH = 'data/london-wards-2014.geojson'

with open(GEO_DATA_PATH) as f:
    geo_london = json.load(f)

year_grocery = pd.read_csv("data/year_osward_grocery.csv")

year_grocery["nutripoints"] = year_grocery.apply(calculate_nutripoints, axis=1)

wellbeing_scores = pd.read_excel(
    "data/london-ward-well-being-probability-scores.xls", sheet_name="Scores", header=[0, 1])
wellbeing_scores = wellbeing_scores.loc[:, (slice(
    None), [2013, "New ward code", "Ward name", "Borough"])].dropna(how="all")
wellbeing_scores = wellbeing_scores.droplevel(1, axis=1)

wellbeing_grocery = pd.merge(
    left=year_grocery, right=wellbeing_scores, left_on='area_id', right_on="New ward code")

LONDON_CENTER_COORD = {"lat": 51.5074, "lon": -0.118092}

In [3]:
FEATURES = ['representativeness_norm', 'fibre',
            'protein', 'fat', 'carb', 'salt', 'nutripoints']

map_plots = []

for f in FEATURES:
    map_plots.append(go.Choroplethmapbox(
        geojson=geo_london,
        locations=wellbeing_grocery['area_id'],
        featureidkey="properties.gss_code_ward",
        text=wellbeing_grocery['Ward name'],
        z=wellbeing_grocery[f],
        colorbar=dict(thickness=20, ticklen=3),
        marker_line_width=0, marker_opacity=0.7,
        visible=False,
        hovertemplate="<b>ward</b>: <b>%{text}</b><br><b>val</b>: %{z}<br><extra></extra>"
    ))

map_plots[0]['visible'] = True

In [4]:
layout = go.Layout(
    autosize=True,
    mapbox=dict(
        style="carto-positron",
        center=LONDON_CENTER_COORD,
        zoom=8.7),
    margin=dict(l=0, r=0, t=0, b=0),
    paper_bgcolor='rgb(204, 204, 204)',
    plot_bgcolor='rgb(204, 204, 204)',
)

In [5]:
# add a dropdown menu in the layout
layout.update(
    updatemenus=[
        dict(x=0.005,
             y=0.95,
             xanchor='left',
             yanchor='middle',
             buttons=[
                 dict(
                     args=['visible', [True, False, False,
                                       False, False, False, False]],
                     label='Representativeness',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, True, False,
                                       False, False, False, False]],
                     label='Fibre',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, True,
                                       False, False, False, False]],
                     label='Protein',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False,
                                       True, False, False, False]],
                     label='Fat',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False,
                                       False, True, False, False]],
                     label='Carbs',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False,
                                       False, False, True, False]],
                     label='Salt',
                     method='restyle'
                 ),
                 dict(
                     args=['visible', [False, False, False,
                                       False, False, False, True]],
                     label='Nutripoints',
                     method='restyle'
                 )
             ])
    ]
);

In [6]:
fig = go.Figure(data=map_plots, layout=layout)

fig.write_html("map.html")

# fig.show()